In [ ]:
import pandas as pd
import numpy as np
from copy import deepcopy
import json
import matplotlib.pyplot as plt
import datetime
import shutil
import os
import re
import random

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
import keras
import pickle

import requests
from google.cloud import storage
from bs4 import BeautifulSoup

In [ ]:
# Make sure the following environment variable is set! 
creds_path = os.environ['GOOGLE_APPLICATION_CREDENTIALS_PATH']
gcs_client = storage.Client.from_service_account_json(creds_path)
bucket = gcs_client.get_bucket('covid-19-forecaster-data')

In [ ]:
# Arguments
lookahead = 8 # Lookahead weeks. Set to 8 by default.
colour_limit = 0.002 # Colour up to 200 in 100k people, above is max

In [ ]:
# Important dates
begin_date_str = '1/27/20' # Make sure it's a monday, and the day before exists in the data
begin_date = datetime.datetime.strptime(begin_date_str, "%m/%d/%y").date()
print(f'Begin date : {begin_date_str}')

today = datetime.date.today()
today_date = f'{today.month}/{today.day}/{today.year-2000}'
print(f'Date today : {today_date}')

end_date = today - datetime.timedelta(today.weekday()+1)
end_date_str = f'{end_date.month}/{end_date.day}/{end_date.year-2000}'
print(f'End of last week : {end_date_str}')

In [ ]:
# Create output folder
week_begin = today - datetime.timedelta(today.weekday())
week_end = week_begin + datetime.timedelta(days=6)
cur_week_name = f'{week_begin.month}-{week_begin.day}-{week_begin.year-2000}-to-{week_end.month}-{week_end.day}-{week_end.year-2000}'
preds_folder = f'./preds/weekly/{cur_week_name}'
output_folder = f'./output/weekly/{cur_week_name}'
print(f'Current identifier: *{cur_week_name}*')

In [ ]:
# Read data
with open(f"{output_folder}/pred_vars.pkl","rb") as f:
    pred_vars = pickle.load(f)
std_val = pred_vars['std_val']
base_cols = pred_vars['base_cols']
combined_data = pred_vars['combined_data']
df_select = pred_vars['df_select']
input_steps = pred_vars['input_steps']
layer_count = pred_vars['layer_count']
unit_count = pred_vars['unit_count']
dropout = pred_vars['dropout']
best_model_path = pred_vars['best_model_path']

In [ ]:
# Read country name - CCODE mapping
ccode_mapping = {}
for _, row in df_select.iterrows():
    if (str(row['CCODE']) != '-1'):
        ccode_mapping[row['Country']] = row['CCODE']

In [ ]:
for itr in range(len(df_select)):
    if str(df_select.loc[itr,'CCODE']) == '-1':
        df_select.loc[itr,'CCODE'] = ccode_mapping[df_select.loc[itr,'Country']]

In [ ]:
df_select.head(3)

In [ ]:
def create_model(input_shape, layer_count, units, dropout, training=None):
    # Layer count check
    if layer_count < 2:
        return None
    
    # Add the first layer
    inputs = keras.Input(shape=(input_shape, 1))
    x = LSTM(units=units, return_sequences=True)(inputs)
    
    # Add further layers
    for layer in range(1, layer_count):
        # Output layer: set units to 0 and don't return sequences
        if layer == (layer_count-1):
            return_sequences = False
        else:
            return_sequences = True

        # Add LSTM layer
        x = LSTM(units=units, return_sequences=return_sequences)(x)
        x = Dropout(dropout)(x, training=training)

    # Adding the output layer
    outputs = Dense(1, activation='relu')(x)

    # Compiling the model
    model = keras.Model(inputs, outputs)
    model.compile(optimizer = 'adam', loss = 'mean_squared_error')
    return model

# Inference

In [ ]:
# Load weights
ensemble_model = True
if ensemble_model:
    test_dropout = 0.2
    test_trials = 100
    training = True
else:
    test_dropout = dropout
    test_trials = 1
    training = None

In [ ]:
all_preds = []
# Load model
model = create_model(input_steps, layer_count, unit_count, test_dropout, training=training)
model.load_weights(best_model_path)

for trial in range(test_trials):
    print(f'Predictions (no. {trial}) ')
    trial_preds = []
    cur_data = combined_data.values[:, -input_steps:]
    
    # Get preds for all future days
    for day in range(lookahead):
        tmp_data = np.reshape(cur_data, (cur_data.shape[0], cur_data.shape[1], 1))
        preds = model.predict(tmp_data)
        cur_data = np.concatenate([cur_data, preds], axis=1)[:, -input_steps:]
        trial_preds.append(preds)
    trial_preds = np.stack(trial_preds, axis=1)
    all_preds.append(trial_preds)
all_preds = np.concatenate(all_preds, axis=2)

In [ ]:
# Get the preds and the data in the original input space
all_preds_norm = all_preds * std_val
combined_data_norm = combined_data * std_val

In [ ]:
# Calculate mean and standard deviation (measurement of uncertainty)
mean_preds = all_preds_norm.mean(axis=2)
std_preds = all_preds_norm.std(axis=2)

In [ ]:
print(all_preds_norm.mean(), std_preds.mean())

In [ ]:
real_data = combined_data_norm.values
full_data = np.concatenate([combined_data_norm.values, mean_preds], axis=1)
full_data_std = np.concatenate([np.zeros(real_data.shape), std_preds], axis=1)

# Visualize the predictions

In [ ]:
# Get week names to visualize
total_weeks_to_show = full_data.shape[1]
week_names = []
cur_date = begin_date
for itr in range(total_weeks_to_show):
    week_begin = cur_date
    week_end = cur_date + datetime.timedelta(days=6)
    week_descr = f'{week_begin.month}/{week_begin.day}/{week_begin.year-2000}-{week_end.month}/{week_end.day}/{week_end.year-2000}'
    week_names.append(week_descr)
    cur_date = cur_date + datetime.timedelta(days=7)

In [ ]:
month_names = {'*1/': 'Jan ', '*2/': 'Feb ', '*3/': 'Mar ', '*4/': 'Apr ', '*5/': 'May ', '*6/': 'Jun ', \
               '*7/': 'Jul ', '*8/': 'Aug ', '*9/': 'Sep ', '*10/': 'Oct ', '*11/': 'Nov ', '*12/': 'Dec '}

In [ ]:
show_cols = 15
week_names_to_show = week_names[-show_cols:]
shown_ticks = [x.replace('/20/', '') for x in week_names_to_show]
shown_ticks = ['*' + (x.replace('/20', '').split('-')[0]) for x in week_names_to_show]
for key in month_names:
    shown_ticks = [x.replace(key, month_names[key]) for x in shown_ticks]

In [ ]:
shown_ticks

In [ ]:
df_select['graph_name'] = ""
for itr in range(len(df_select)):
    df_select.loc[itr, 'graph_name'] = ''.join(e for e in df_select.loc[itr, 'Name'].lower() if e.isalnum()) + \
                              '-' + \
                              ''.join(e for e in df_select.loc[itr, 'Country'].lower() if e.isalnum())

In [ ]:
# NEW STYLE SIMPLE PREDICTIONS

buffer = 0.2
limit = 20
above_limit = 10
ybuffer = 3
future_weeks = full_data.shape[1] - real_data.shape[1]
if os.path.exists(f'{preds_folder}/pred_plots'):
    shutil.rmtree(f'{preds_folder}/pred_plots')
os.makedirs(f'{preds_folder}/pred_plots')
for idx in range(len(df_select)):
    if df_select.loc[idx, 'is_country'] == 0:
        country_name = df_select.loc[idx, 'graph_name']
        code_name = country_name
    else:
        country_name = df_select.loc[idx, 'Country']
        code_name = df_select.loc[idx, 'CCODE']
    print('Saving ', code_name)
    data_to_show = np.maximum(0.0, 100000*full_data[idx, -show_cols:])
    std_to_show = np.maximum(0.0, 100000*full_data_std[idx, -show_cols:])
    real_data_to_show = data_to_show[:-future_weeks]

    plt.figure(figsize=(5, 3))
    plt.title(f'Weekly cases per 100k people')
    plt.plot([x for x in range(len(data_to_show))], data_to_show, 'k:o', markersize=5, linewidth=2)
    plt.fill_between([x for x in range(len(data_to_show))], (data_to_show-std_to_show), (data_to_show+std_to_show), color='k', alpha=.05)
    plt.fill_between([x for x in range(len(data_to_show))], (data_to_show-2*std_to_show), (data_to_show+2*std_to_show), color='k', alpha=.025)
    plt.plot([x for x in range(len(real_data_to_show))], real_data_to_show, 'k-o', markersize=5, linewidth=2)
    
#     plt.legend(['our predictions', 'past data'], loc='upper left')

    plt.plot([0, show_cols-1], [limit, limit], 'r--')
    plt.plot(len(real_data_to_show), data_to_show[len(real_data_to_show)], 'm*', markersize=20)
    plt.xticks([x for x in range(show_cols)], shown_ticks, rotation=60, ha='center')
    
    max_y = max(limit+above_limit, np.ceil(max(data_to_show+2*std_to_show) * 1.2))+ybuffer
    plt.ylim([-ybuffer, max_y])
    plt.xlim([-buffer, (show_cols-1)+buffer])
    if data_to_show[7] > max_y * 0.5:
        plt.text(7-0.8, data_to_show[7]-(max_y/3.2), ' this\nweek', fontsize=12, color='b', bbox=dict(facecolor='b', alpha=0.1))
    else:
        plt.text(7-0.8, data_to_show[7]+(max_y/6.5), ' this\nweek', fontsize=12, color='b', bbox=dict(facecolor='b', alpha=0.1))
    plt.text(1.1, max_y*0.85, 'recent cases', fontsize=12, color='k', alpha=0.6, bbox=dict(facecolor='black', alpha=0.1))
    plt.text(8, max_y*0.85, 'future predictions', fontsize=12, color='k', alpha=0.6, bbox=dict(facecolor='black', alpha=0.05))
    plt.text(2.75, limit+(max_y/20), 'risk threshold (20 per 100k)', fontsize=12, color='r', alpha=0.6, bbox=dict(facecolor='red', alpha=0.1))
    plt.xlabel('weeks starting with')
    plt.ylabel('number of cases')
    
    plt.text(11, max_y*1.13, 'covidtripplanner.com',
         fontsize=7, color='gray',
         ha='left', va='bottom', alpha=0.6)
    
    # Draw lines between the week dots and markers
    for itr in range(len(week_names_to_show)):
        plt.plot([itr, itr], [0, data_to_show[itr]], 'k:', alpha=0.05)
    
    plt.gcf().subplots_adjust(bottom=0.28, left=0.15, right=0.96)
    plt.savefig(f'{preds_folder}/pred_plots/{code_name}.jpg', dpi=200)
    plt.show()
    plt.close()

In [ ]:
# OLD, COMPLICATED PREDICTIONS

# show_cols = 20
# buffer = 0.2
# limit = 20
# ybuffer = 0.5
# future_weeks = full_data.shape[1] - real_data.shape[1]
# if os.path.exists(f'{preds_folder}/pred_plots'):
#     shutil.rmtree(f'{preds_folder}/pred_plots')
# os.makedirs(f'{preds_folder}/pred_plots')
# for idx in range(len(df_select)):
#     if df_select.loc[idx, 'is_country'] == 0:
#         country_name = df_select.loc[idx, 'Name'] + ' ' + df_select.loc[idx, 'CCODE']
#         code_name = country_name
#     else:
#         country_name = df_select.loc[idx, 'Country']
#         code_name = df_select.loc[idx, 'CCODE']
#     data_to_show = np.maximum(0.0, 100000 * full_data[idx, -show_cols:])
#     std_to_show = np.maximum(0.0, 100000 * full_data_std[idx, -show_cols:])
#     week_names_to_show = week_names[-show_cols:]
#     real_data_to_show = data_to_show[:-future_weeks]
#     real_week_names_to_show = week_names_to_show[:-future_weeks]

#     plt.figure(figsize=(10, 5))
#     plt.title(f'Weekly Covid-19 data for {country_name}')
#     plt.plot([x for x in range(len(data_to_show))], data_to_show, 'c--o')
#     plt.fill_between([x for x in range(len(data_to_show))], (data_to_show-std_to_show), (data_to_show+std_to_show), color='c', alpha=.1)
#     plt.fill_between([x for x in range(len(data_to_show))], (data_to_show-2*std_to_show), (data_to_show+2*std_to_show), color='c', alpha=.05)
#     plt.plot([x for x in range(len(real_data_to_show))], real_data_to_show, 'b-o')
#     plt.plot([0, show_cols-1], [limit, limit], 'r--')
#     plt.plot(len(real_data_to_show), data_to_show[len(real_data_to_show)], 'm*', markersize=14)
#     plt.ylabel('weekly cases per 100k')
#     shown_ticks = [x for x in range(0, len(week_names_to_show), 1)]
#     shown_xlabels = [week_names_to_show[x] for x in shown_ticks]
#     plt.xticks(shown_ticks, shown_xlabels, rotation=45, ha='right')
#     plt.xlabel('week')
#     plt.ylim([-ybuffer, max(limit+5, np.ceil(max(data_to_show+2*std_to_show)) + 5)+ybuffer])
#     plt.xlim([-buffer, (show_cols-1)+buffer])
#     plt.legend(['predicted', 'real', f'{limit} cases per 100k', f'This week - [{week_names_to_show[len(real_data_to_show)]}] (predicted)'], loc='upper left')
    
#     # Draw lines between the week dots and markers
#     for itr in range(len(week_names_to_show)):
#         plt.plot([itr, itr], [0, data_to_show[itr]], 'b:', alpha=0.1)
    
#     plt.gcf().subplots_adjust(bottom=0.3, left=0.1, right=0.96)
#     plt.savefig(f'{preds_folder}/pred_plots/{code_name}.jpg', dpi=200)
#     plt.show()
#     plt.close()

## Make risk predictions for all countries and regions

In [ ]:
# Get risk preds (most likely scenario)
risk_preds = (all_preds_norm).mean(2)
risk_preds[risk_preds > colour_limit] = colour_limit
risk_preds[risk_preds < 0] = 0
risk_preds = np.round((risk_preds/colour_limit) * 100).astype(int)

## Save outputs and configuration.

In [ ]:
df_final = deepcopy(df_select[base_cols + ['graph_name']])
for itr in range(len(week_names)):
    df_final.loc[:, week_names[itr]] = full_data[:, itr]

In [ ]:
df_final.to_csv(f'{output_folder}/df_final.csv')

In [ ]:
# Write individual predictions to file
with open(f"{output_folder}/all_preds.pkl","wb") as f:
    pickle.dump(all_preds_norm,f)
f"{output_folder}/all_preds.pkl"

In [ ]:
# Write individual predictions to file
with open(f"{output_folder}/risk_preds.pkl","wb") as f:
    pickle.dump(risk_preds,f)
f"{output_folder}/risk_preds.pkl"

In [ ]:
config = {}
config['lookahead'] = lookahead
config['week_names'] = week_names
config['current_week'] = week_names[-lookahead]
with open(f"{output_folder}/config.pkl","wb") as f:
    pickle.dump(config,f)
f"{output_folder}/config.pkl"

## Save plots to the cloud

In [ ]:
# Delete this week's predictions if they are up there
blobs = bucket.list_blobs(prefix=f"preds/weekly/{cur_week_name}/pred_plots")
for blob in blobs:
    blob.delete()

In [ ]:
# Upload this week's predictions
for path, subdirs, files in os.walk(preds_folder + '/pred_plots'):
    path = path.replace("\\","/")
    directory_name = path.replace(preds_folder, f"preds/weekly/{cur_week_name}")
    for file in files:
        blob = bucket.blob(directory_name+'/'+file)
        blob.upload_from_filename(os.path.join(path, file))

## Save plots to the "current" folder on the cloud

In [ ]:
# Delete this week's predictions if they are up there
blobs = bucket.list_blobs(prefix=f"preds/weekly/current/pred_plots")
for blob in blobs:
    blob.delete()

In [ ]:
# Upload this week's predictions
for path, subdirs, files in os.walk(preds_folder + '/pred_plots'):
    path = path.replace("\\","/")
    directory_name = path.replace(preds_folder, f"preds/weekly/current")
    for file in files:
        blob = bucket.blob(directory_name+'/'+file)
        blob.upload_from_filename(os.path.join(path, file))
        blob.make_public()